# Assignment 3

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        0.0  0.0  4.0  4.0  4.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        4.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 2        0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
 3        4.0  0.0  5.0  0.0  1.0  0.0  3.0  2.0  0.0  0.0  ...  0.0  0.0  0.0   
 4        3.0  0.0  4.0  0.0  0.0  3.0  2.0  2.0  0.0  5.0  ...  0.0  4.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  4.0  5.0  ...  0.0  0.0  4.0   
 296      0.0  0.0  5.0  4.0  0.0  1.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
 297      4.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 298      0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()


# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [7]:
## Put all your implementation for your solution in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9

#number of users and items(reffered from week 11 tute)
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
n_prediction_user = np.zeros((n_users, n_users))

for i, user_i_vec in enumerate(imputed_train_ds):
    for j, user_j_vec in enumerate(imputed_train_ds):

        # ratings corated by the current pair of users
        mask_i_user = user_i_vec > 0
        mask_j_user = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index_user = np.intersect1d(np.where(mask_i_user), np.where(mask_j_user))
        if len(corrated_index_user) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson correlation for user
        user_i_sub_mean = user_i_vec[corrated_index_user] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index_user] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim_user = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting for the user
        weighted_sim_user = (min(len(corrated_index_user), GAMMA) / GAMMA) * sim_user
        
        n_prediction_user[i][j] = weighted_sim_user
        

            

n_prediction_items = np.zeros((n_items, n_items))

for i, item_i_vec in enumerate(imputed_train_ds.T):
    for j, item_j_vec in enumerate(imputed_train_ds.T):

        # ratings corated by the current pair of items
        mask_i_item = item_i_vec > 0
        mask_j_item = item_j_vec > 0

        # corrated index, skip if there are none
        corrated_index_item = np.intersect1d(np.where(mask_i_item), np.where(mask_j_item))
        if len(corrated_index_item) == 0:
            continue

        # average value of item_i_vec and item_j_vec
        mean_item_i = np.sum(item_i_vec) / (np.sum(np.clip(item_i_vec, 0, 1)) + EPSILON)
        mean_item_j = np.sum(item_j_vec) / (np.sum(np.clip(item_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr fro item
        item_i_sub_mean = item_i_vec[corrated_index_item] - mean_item_i
        item_j_sub_mean = item_j_vec[corrated_index_item] - mean_item_j

        r_ui_sub_ri_sq = np.square(item_i_sub_mean)
        r_uj_sub_rj_sq = np.square(item_j_sub_mean)

        r_ui_sub_ri_sq_sum_sqrt = np.sqrt(np.sum(r_ui_sub_ri_sq))
        r_uj_sub_rj_sq_sum_sqrt = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(item_i_sub_mean * item_j_sub_mean) / (r_ui_sub_ri_sq_sum_sqrt * r_uj_sub_rj_sq_sum_sqrt + EPSILON)

        # significance weighting for item
        weighted_sim = (min(len(corrated_index_item), DELTA) / DELTA) * sim
        
        n_prediction_items[i][j] = weighted_sim
        
        

missing_data = np.zeros((n_users, n_items))

for (i, j), rating in np.ndenumerate(imputed_train_ds):
           
    if rating == 0:
                                                
        # get the similar user under threshold ITA
        similar_user = np.zeros(n_users)
        for k in range(1, len(n_prediction_user[i])):
            if n_prediction_user[i][k] > ITA:
                similar_user[k-1] = imputed_train_ds[i][k]
                
        similar_user = similar_user[similar_user != 0]
         
        # get the similar item under threshold THETA   
        similar_items = np.zeros(n_items)
        for k in range(1, len(n_prediction_items[i])):
            if n_prediction_items[i][k] > THETA:
                similar_items[k-1] = imputed_train_ds[i][k]
                
        similar_items = similar_items[similar_items != 0]
                
        # get positions of similar users and items by taking the length of similar user and item 
        sim_users_pos = np.argsort(n_prediction_user[i])[-(len(similar_user)+1):-1]
        sim_items_pos = np.argsort(n_prediction_items[j])[-(len(similar_items)+1):-1]        

        # get mean of current user and mean of similar users
        sim_users = imputed_train_ds[sim_users_pos]
        similar_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)
        current_user_mean = np.sum(imputed_train_ds[i]) / (np.sum(np.clip(imputed_train_ds[i], 0, 1)) + EPSILON)       
        
         # get mean of current item and mean of similar item
        sim_items = imputed_train_ds.T[sim_items_pos]
        similar_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)
        current_item_mean = np.sum(imputed_train_ds.T[j]) / (np.sum(np.clip(imputed_train_ds.T[j], 0, 1)) + EPSILON)
        
        # calculate and sum PCC * sim_user - sim_user_mean
        #calculate and sum PCC * sim_item - sim_item_mean
        users_flag_item = sim_users[:, i] > 0
        sub_user_sum= similar_user[users_flag_item] * (sim_users[users_flag_item, j] - similar_user_mean[users_flag_item])
        sub_item_sum = similar_items * (sim_items[:, i] - similar_item_mean)

        # discard the not rated item and user
        sub_user_sum= sub_user_sum*np.clip(sim_users[users_flag_item, j], 0, 1)
        sub_item_sum=sub_item_sum* np.clip(sim_items[:, i], 0, 1)
        
        #equation 9
        user_pred= current_user_mean + (np.sum(sub_user_sum) / (np.sum(sim_users[users_flag_item]) + EPSILON))
        #equation 10
        item_pred = current_item_mean + (np.sum(sub_item_sum) / (np.sum(sim_items) + EPSILON))
        #equation 8
        predict = LAMBDA * user_pred+ (LAMBDA-1) * item_pred
        
        flag_user = False
        
        if len(sim_users) > 0:
            flag_user = True
            
        flag_item = False
        if len(sim_items) > 0:
            flag_item = True
        # if both similar itema nd user is available
        if flag_user == True and flag_item == True:               
            missing_data[i][j] = predict
        #if similar user is available and similar item not available
        elif flag_user == True and flag_item==False:
            missing_data[i][j] =user_pred
        #if similar item is available and similar useer not available
        elif flag_item == True and flag_user==False:
            missing_data[i][j] = item_pred
        
       
            

    else: 
        missing_data[i][j] = rating
        
#data with the rating we predicted    
for i in range(len(imputed_train_ds)):       
        for j in range(len(imputed_train_ds[i])):
            imputed_train_ds[i][j]=missing_data[i][j]
            imputed_train_ds[i][j] = np.clip(imputed_train_ds[i][j], 0, 5)

# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [8]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.000000,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,4.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4.000000,3.962500,3.962500,2.000000,3.962500,3.962500,3.962500,3.962500,3.962500,3.962500,...,3.962500,3.962500,3.962500,3.962500,3.962500,3.962500,3.962500,3.962500,3.962500,3.962500
2,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4.000000,0.000000,5.000000,0.000000,1.000000,0.000000,3.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000
4,3.000000,3.461538,4.000000,3.840728,3.472727,3.000000,2.000000,2.000000,2.998258,5.000000,...,2.837209,4.000000,3.608696,3.724693,3.019608,2.342857,2.342105,3.523348,2.903226,3.209425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.143617,1.337070,1.126047,1.222253,1.333714,1.354944,1.198909,5.000000,4.000000,5.000000,...,1.515233,1.467114,4.000000,1.258291,4.000000,2.000000,3.000000,1.316157,1.504564,1.414254
296,0.000000,0.000000,5.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
297,4.000000,3.545455,3.000000,3.545455,3.545455,3.545455,3.545455,3.545455,3.545455,3.545455,...,3.545455,3.545455,3.545455,3.545455,3.545455,3.545455,3.545455,3.545455,3.545455,3.545455
298,4.328358,4.328358,4.328358,5.000000,4.328358,4.328358,4.328358,4.328358,4.328358,4.328358,...,4.328358,4.328358,4.328358,4.328358,4.328358,4.328358,4.328358,4.328358,4.328358,4.328358


In [9]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        #print(corrated_index)
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[-2.96443237e-01, -1.91698625e-01,  2.31993585e-01, ...,
         1.00780025e-01,  1.55719289e-01, -1.53743248e-01],
       [ 2.64327438e-02,  2.61879525e-02, -8.29403738e-02, ...,
         2.26497439e-01, -7.98579280e-02,  1.11052884e-01],
       [ 3.75717734e-01, -1.98880299e-01,  1.88926244e-01, ...,
        -2.76348435e-01,  6.80463221e-03,  9.99999996e-02],
       ...,
       [-3.55443546e-01, -3.62229098e-03,  1.70301694e-01, ...,
        -1.06017763e-01,  3.41944985e-15, -6.61578810e-02],
       [ 3.23175599e-01,  8.25652197e-02,  8.63165931e-02, ...,
         3.36480936e-01, -9.60618131e-02, -1.99999998e-01],
       [-4.06115745e-01, -1.61321735e-01,  7.47867435e-01, ...,
         1.65680374e-01, -5.27701818e-02,  2.53238355e-01]])

## Predict Ratings of Testing Set

In [10]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0.        , 0.        , 0.        , ..., 0.        , 2.87838242,
        0.        ],
       [0.        , 3.79035552, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.88720088, 0.        , 3.83041398, ..., 3.3986045 , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 5.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 3.21534289, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [11]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7481804864185274, RMSE: 0.9657759293902441
